In [1]:
spark

In [2]:
# Set this bucket path to where your csv files are
bucket_path = 'gs://my-project-bucket-gr/landing/D1/'

In [3]:
# Get a collection of objects in the bucket
file_objects = sc.wholeTextFiles(bucket_path + '*.csv')

# file_object will consists of an RDD with filename is the 'key' and the file contents as the 'value'

In [4]:
# Create a file name list from the file objects (this will take about 5 minutes)
filename_list = file_objects.keys().distinct().collect()

# This will produce a list with names like: 'gs://my-project-bucket-gr/landing/D1/A.US_D1.csv'

# Remove the bucket path portion from the list of files so we end up with filenames like 'A.US_D1.csv'
filename_list = [fn.replace(bucket_path, '') for fn in filename_list]

In [5]:
# Show the list of file names
print(filename_list)

['A.US_D1.csv', 'AAL.US_D1.csv', 'ABBV.US_D1.csv', 'AAPL.US_D1.csv', 'ACGL.US_D1.csv', 'ACN.US_D1.csv', 'ADSK.US_D1.csv', 'AEE.US_D1.csv', 'AES.US_D1.csv', 'AIG.US_D1.csv', 'AKAM.US_D1.csv', 'ALGN.US_D1.csv', 'ALL.US_D1.csv', 'ALLE.US_D1.csv', 'AMAT.US_D1.csv', 'AMD.US_D1.csv', 'AMT.US_D1.csv', 'AMZN.US_D1.csv', 'ANSS.US_D1.csv', 'AON.US_D1.csv', 'AOS.US_D1.csv', 'APA.US_D1.csv', 'APH.US_D1.csv', 'ARE.US_D1.csv', 'AVGO.US_D1.csv', 'AWK.US_D1.csv', 'AXP.US_D1.csv', 'AZN.US_D1.csv', 'AZO.US_D1.csv', 'BA.US_D1.csv', 'BALL.US_D1.csv', 'BAX.US_D1.csv', 'BBWI.US_D1.csv', 'BIIB.US_D1.csv', 'BK.US_D1.csv', 'BKNG.US_D1.csv', 'BLDR.US_D1.csv', 'BRK.B.US_D1.csv', 'BXP.US_D1.csv', 'C.US_D1.csv', 'CAG.US_D1.csv', 'CARR.US_D1.csv', 'CB.US_D1.csv', 'CBRE.US_D1.csv', 'CCEP.US_D1.csv', 'CCI.US_D1.csv', 'CCL.US_D1.csv', 'CDW.US_D1.csv', 'CF.US_D1.csv', 'CFG.US_D1.csv', 'CI.US_D1.csv', 'CINF.US_D1.csv', 'CLX.US_D1.csv', 'CMA.US_D1.csv', 'CMCSA.US_D1.csv', 'CME.US_D1.csv', 'CMS.US_D1.csv', 'COF.US_D1.csv'

In [6]:
# Provide a list of column that we are interested in
feature_columns = ['stock_symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume', 
                    'bbands_20_upperband', 'bbands_20_middleband', 
                    'bbands_20_lowerband', 'dema_20', 'ema_20', 'ema_60', 
                    'kama_20', 'kama_60', 'ma_20', 'ma_60', 
                    'sma_20', 'sma_60', 
                    'tema_20', 'tema_60', 'adx_20', 'adx_60', 
                    'macd_20_40_10_macd', 'macd_60_120_30_macd', 
                    'rsi_20', 'rsi_60']

In [7]:
#output_path = 'gs://my-project-bucket-gr/cleaned/'
# Read the Parquet file from GCS    parquet_path = "gs://my-project-bucket-gr/cleaned/stock_data_A.parquet"

# this was just for testing output_path = 'gs://my-new-bucket-xx/cleaned/'
output_path = 'gs://my-project-bucket-gr/cleaned/stock_data_*.parquet'

#stock_data_*.parquet


#######################################################

In [8]:
from pyspark.sql.functions import regexp_replace, col, lit, lead
from pyspark.sql.window import Window

# Loop through all of the files in the filename list 
for filename in filename_list:
    print("Working on filename: ", filename)
    # Read in a file to a spark data frame
    sdf = spark.read.csv(bucket_path + filename, inferSchema=True, header=True)
    # Add the stock symbol as a new column. Base this off of the filename such as "AAPL.US_D1.csv"
    sdf = sdf.withColumn("stock_symbol", lit(filename))
    # Remove the "US_D1.csv" from the stock symbol
    sdf = sdf.withColumn("stock_symbol", regexp_replace(col("stock_symbol"), "\.US_D1\.csv", ""))
    # Select the featured columns only
    sdf = sdf.select(feature_columns)
    windowSpec = Window.partitionBy("stock_symbol").orderBy("datetime")
    sdf = sdf.withColumn("next_day_close", lead("close", 1).over(windowSpec))
    # Remove records with nulls
    sdf = sdf.na.drop()
    # Save this dataframe as a Parquet file in the /cleaned folder
    output_filename = output_path + filename + '.parquet'
    sdf.write.mode("overwrite").parquet(output_filename)



Working on filename:  A.US_D1.csv


24/05/13 16:04:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Working on filename:  AAL.US_D1.csv


Working on filename:  ABBV.US_D1.csv


Working on filename:  AAPL.US_D1.csv


...

Working on filename:  WFC.US_D1.csv


Working on filename:  WHR.US_D1.csv


Working on filename:  WMB.US_D1.csv


Working on filename:  ZBH.US_D1.csv


Working on filename:  ZION.US_D1.csv
